<a href="https://colab.research.google.com/github/aquamarine-28/Reality-Check/blob/main/reality_check_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%pip install ultralytics

import os
import tensorflow as tk
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
import numpy as np
import cv2
import ultralytics
from ultralytics import YOLO


ultralytics.checks()

train_dir = 'kaggle/input/deepfake-and-real-images/Dataset/Train'
validation_dir = 'kaggle/input/deepfake-and-real-images/Dataset/Validation'
test_dir = 'kaggle/input/deepfake-and-real-images/Dataset/Test'

# Preprocessing the images
model = YOLO()

# Function to extract face ROI from an image using YOLO
def extract_face(img_path):

# Perform object detection
    results = model(img_path)
    boxes = results[0].boxes
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    roi = None

    if len(boxes) > 0:
        for i, box in enumerate(boxes):
            top_left_x = int(box.xyxy.tolist()[0][0])
            top_left_y = int(box.xyxy.tolist()[0][1])
            bottom_right_x = int(box.xyxy.tolist()[0][2])
            bottom_right_y = int(box.xyxy.tolist()[0][3])

        # Crop face region
            roi = img[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
        return roi
    else:
        return None

# Additional variables like clip_limit and grid_size are defined outside the function.
clip_limit = 2.0
grid_size = (8,8)

# Define data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Use datagen.flow_from_directory() to load images from the directory
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='binary',
    classes=['Fake', 'Real']
)
validation_dir = datagen.flow_from_directory(
    validation_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='binary',
    classes=['Fake', 'Real']
)
test_dir = datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='binary',
    classes=['Fake', 'Real']
)
# Define the model
model = Sequential()

# Convolutional Layer 1
model.add(Conv2D(64, (3,3),
                 activation='relu',
                 padding='same',
                 input_shape=(224,224,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

# Convolutional Layer 2
model.add(Conv2D(64, (3,3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Full Connected Layer
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Display the model summary
model.summary()

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])






Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.9/107.7 GB disk)
Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 224, 224, 64)      256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 74, 74, 64)        0  